In [1]:
from GMLQueryRewriter import KG_Meta
kg = KG_Meta.KgMeta()

In [3]:
#List available GML Tasks
query = """ 
Select ?s where {?s a <kgnet:types/GML_Task>.}"""
kg.query(query)

,s
0,kgnet:GML/GML_Task/DBLP_PV
1,kgnet:GML/GML_Task/MAG_PV
2,kgnet:GML/GML_Task/CIS_FD
3,kgnet:GML/GML_Task/YAGO_CA
4,kgnet:GML/GML_Task/DBLP_AF


In [4]:
#List available models for a task
query = """  select ?m
 where {
 <kgnet:GML/GML_Task/DBLP_PV> <kgnet:GML_ID> ?m .}"""
kg.query(query)

,m
0,kgnet:GML/GML_Model/ID/1111111
1,kgnet:GML/GML_Model/ID/1111112
2,kgnet:GML/GML_Model/ID/1111113
3,kgnet:GML/GML_Model/ID/1111114


In [5]:
#List model attributes
query = """ select ?m ?p ?o
 where {
 <kgnet:GML/GML_Task/DBLP_PV> <kgnet:GML_ID> ?m .
 ?m a <kgnet:types/GML_Model>.
 ?m <kgnet:GMLMethod> <kgnet:GML/Method/G-SAINT>.
 ?m ?p ?o .
 }"""
kg.query(query)

,m,p,o
0,kgnet:GML/GML_Model/ID/1111111,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,kgnet:types/GML_Model
1,kgnet:GML/GML_Model/ID/1111111,kgnet:GMLMethod,kgnet:GML/Method/G-SAINT
2,kgnet:GML/GML_Model/ID/1111111,kgnet:InferenceTime,1
3,kgnet:GML/GML_Model/ID/1111111,kgnet:ModelAccuracy,47
4,kgnet:GML/GML_Model/ID/1111111,kgnet:Sampler,kgnet:GML/GNNSampler/RW
5,kgnet:GML/GML_Model/ID/1111111,kgnet:TaskBudget,kgnet:Task/TaskBudget
6,kgnet:GML/GML_Model/ID/1111111,kgnet:API_URL,http://127.0.0.1:64646/all


/home/afandi/GitRepos/KGNET_DEV/KGNET


In [13]:
# DBLP Paper Venue prediction queries

from GMLQueryRewriter import gmlRewriter as gr

query = """
prefix dblp:<https://dblp.org/rdf/schema#>
prefix kgnet: <https://www.kgnet.ai/>
select ?title ?venue 
where {
?paper a dblp:Publication.
?paper dblp:title ?title.
?paper <https://dblp.org/rdf/schema#publishedIn> ?o.
?paper <https://dblp.org/has_gnn_model> 1.

?paper ?NodeClassifier ?venue.

?NodeClassifier a <kgnet:types/NodeClassifier>.
?NodeClassifier <kgnet:GML/TargetNode> <dblp:paper>.
?NodeClassifier <kgnet:GML/NodeLabel> <dblp:venue>.
}
limit 10
"""
dataQuery,gmlQuery = gr.execute(query)

In [14]:
print(gmlQuery)

PREFIX dblp: <https://dblp.org/rdf/schema#>
PREFIX kgnet: <https://www.kgnet.ai/>

    #SELECT ?NodeClassifier
    SELECT ?apiUrl
    WHERE
    {?NodeClassifier <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <kgnet:types/NodeClassifier> .
?NodeClassifier <kgnet:GML/TargetNode> <dblp:Publication> .
?NodeClassifier <kgnet:GML/NodeLabel> <dblp:venue> .
?NodeClassifier <kgnet:term/uses> ?gmlModel .
?gmlModel <kgnet:API_URL> ?apiUrl .}


In [15]:
print(dataQuery)

PREFIX dblp: <https://dblp.org/rdf/schema#>
PREFIX kgnet: <https://www.kgnet.ai/>
SELECT ?title,
 sql:getKeyValue_v2(?paper,?venue_dic) as ?venue 

 WHERE { 
?paper <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> dblp:Publication .
?paper dblp:title ?title .
?paper <https://dblp.org/rdf/schema#publishedIn> ?o .
?paper <https://dblp.org/has_gnn_model> 1 .
{SELECT sql:getNodeClass_v2('http://127.0.0.1:64646/all','dblp:Publication') 
 as ?venue_dic WHERE {} }}
 LIMIT 10


In [16]:
# GML as a service

from GMLWebServiceApis import GML_to_CSV as gcsv
import time
import os
import pandas as pd
filename = os.path.join(time.strftime("%Y%m%d-%H%M%S"+'.csv'))
gcsv.sparqlToCSV(dataQuery, filename)
gcsv.mapVenues(filename,r'./GMLWebServiceApis/data/labelidx2labelname.csv')
df = pd.read_csv(filename)
df

,title,venue
0,A data partition method for parallel self-orga...,IJCNN
1,On-line Gaussian mixture density estimator for...,IJCNN
2,System identification with partial-state measu...,IJCNN
3,Relearning procedure to adapt pollutant predic...,IJCNN
4,A dynamic neural network for syllable recognit...,IJCNN
5,EEG based gesture mimicking by an artificial l...,IJCNN
6,Analysis and Implementation of Simple Dynamic ...,IJCNN
7,Expected similarity estimation for large scale...,IJCNN
8,AUC Estimation and Concept Drift Detection for...,IJCNN
9,Attention-driven Multi-sensor Selection.,IJCNN
